<a href="https://colab.research.google.com/github/dfragos/EMOS-2024/blob/main/Spacial/Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install R and IRkernel
!apt-get install r-base
!R -e "install.packages('IRkernel', repos='http://cran.us.r-project.org')"
!R -e "IRkernel::installspec(user = FALSE)"

Colab menu: Runtime > Change runtime type.
Runtime type: R.

In [ ]:
# Install necessary packages if not already installed
if (!requireNamespace("giscoR", quietly = TRUE)) install.packages("giscoR")
if (!requireNamespace("eurostat", quietly = TRUE)) install.packages("eurostat")
if (!requireNamespace("sf", quietly = TRUE)) install.packages("sf")
if (!requireNamespace("ggplot2", quietly = TRUE)) install.packages("ggplot2")
if (!requireNamespace("dplyr", quietly = TRUE)) install.packages("dplyr")

# Load libraries
library(giscoR)
library(eurostat)
library(sf)
library(ggplot2)
library(dplyr)

In [ ]:
# Search for unemployment data in Eurostat
datasets <- search_eurostat("unemployment")
print(datasets)

# Get the dataset ID for monthly unemployment rates
dataset_id <- "une_rt_m"

# Load unemployment data
unemployment_data <- get_eurostat(id = dataset_id, time_format = "date")

# Filter data for the latest available month and only total population
latest_date <- max(unemployment_data$time)
filtered_data <- unemployment_data %>%
  filter(time == latest_date & sex == "T" & age == "TOTAL") %>%
  select(geo, values)

In [ ]:
# Get spatial data for European countries using giscoR
europe_map <- gisco_get_countries(resolution = "10", year = 2020)

# Join the spatial data with unemployment data
europe_unemployment <- europe_map %>%
  left_join(filtered_data, by = c("ISO3_CODE" = "geo"))

In [ ]:
# Create a choropleth map
ggplot(europe_unemployment) +
  geom_sf(aes(fill = values), color = "black", size = 0.1) +
  scale_fill_viridis_c(option = "plasma", name = "Unemployment Rate (%)") +
  labs(
    title = "Unemployment Rates in Europe",
    subtitle = paste("Latest data as of", format(latest_date, "%B %Y")),
    caption = "Source: Eurostat and GISCO"
  ) +
  theme_minimal() +
  theme(
    plot.title = element_text(size = 16, face = "bold"),
    plot.subtitle = element_text(size = 12),
    legend.position = "right"
  )

In [ ]:
# Use eurostat
library(eurostat)
popdens <- get_eurostat("demo_r_d3dens") %>%
  filter(TIME_PERIOD == "2021-01-01")

In [ ]:
# Merge data
nuts3_sf <- nuts3 %>%
  left_join(popdens, by = "geo")

nuts3_sf <- nuts3 %>%
  left_join(popdens, by = c("NUTS_ID" = "geo"))


# Breaks and labels

br <- c(0, 25, 50, 100, 200, 500, 1000, 2500, 5000, 10000, 30000)
labs <- prettyNum(br[-1], big.mark = ",")

# Label function to be used in the plot, mainly for NAs
labeller_plot <- function(x) {
  ifelse(is.na(x), "No Data", x)
}
nuts3_sf <- nuts3_sf %>%
  # Cut with labels
  mutate(values_cut = cut(values, br, labels = labs))


# Palette
pal <- hcl.colors(length(labs), "Lajolla")


# Plot
ggplot(nuts3_sf) +
  geom_sf(aes(fill = values_cut), linewidth = 0, color = NA, alpha = 0.9) +
  geom_sf(data = country_lines, col = "black", linewidth = 0.1) +
  # Center in Europe: EPSG 3035
  coord_sf(
    xlim = c(2377294, 7453440),
    ylim = c(1313597, 5628510)
  ) +
  # Legends
  scale_fill_manual(
    values = pal,
    # Label for NA
    labels = labeller_plot,
    drop = FALSE, guide = guide_legend(direction = "horizontal", nrow = 1)
  ) +
  # Theming
  theme_void() +
  # Theme
  theme(
    plot.title = element_text(
      color = rev(pal)[2], size = rel(1.5),
      hjust = 0.5, vjust = -6
    ),
    plot.subtitle = element_text(
      color = rev(pal)[2], size = rel(1.25),
      hjust = 0.5, vjust = -10, face = "bold"
    ),
    plot.caption = element_text(color = "grey60", hjust = 0.5, vjust = 0),
    legend.text = element_text(color = "grey20", hjust = .5),
    legend.title = element_text(color = "grey20", hjust = .5),
    legend.position = "bottom",
    legend.title.position = "top",
    legend.text.position = "bottom",
    legend.key.height = unit(.5, "line"),
    legend.key.width = unit(2.5, "line")
  ) +
  # Annotate and labs
  labs(
    title = "Population density in 2021",
    subtitle = "NUTS-3 level",
    fill = "people per sq. kilometer",
    caption = paste0(
      "Source: Eurostat, ", gisco_attributions(),
      "\nBased on Milos Popovic: ",
      "https://milospopovic.net/how-to-make-choropleth-map-in-r/"
    )
  )